<a href="https://www.kaggle.com/code/akscent/fitmlforcharn-prediction?scriptVersionId=154080459" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns

from catboost import CatBoostClassifier
from matplotlib import pyplot as plt
from sklearn.compose import ColumnTransformer
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier, StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.tree import DecisionTreeClassifier, plot_tree
from pprint import pprint
from warnings import filterwarnings

import plotly.graph_objs as go
from plotly.subplots import make_subplots
import plotly.express as px

filterwarnings("ignore", category=pd.errors.PerformanceWarning)
filterwarnings("ignore", category=FutureWarning)
filterwarnings("ignore", category=Warning)
filterwarnings("ignore", category=DeprecationWarning)

try:
    import cupy as cp
    gpu_available = True
except ImportError:
    gpu_available = False

n_jobs = -1 if gpu_available else 1

In [ ]:
# import wandb
# #https://wandb.ai/authorize
# wandb.login()

In [ ]:
# CONFIG = dict ()

# CONFIG['model_name'] = 'DJStacking'
# print('Training configuration: ', CONFIG)

# # Initialize W&B run
# run = wandb.init(project='AlfaSibHack',
#                  config=CONFIG,
#                  group='DJStacking', 
#                  job_type='train')


# Load data

In [ ]:
train_df = pd.read_csv('/kaggle/input/yandexcontest/train (2).csv')
test_df = pd.read_csv('/kaggle/input/yandexcontest/train (2).csv')
data = train_df
y_col = ["target"]

# x_train = pd.read_csv('/kaggle/input/sibalfahack/X_train_v1.csv')
# y_train = pd.read_csv('/kaggle/input/sibalfahack/y_train_v1.csv')
# test_df = pd.read_csv('/kaggle/input/sibalfahack/test_v1 (2).csv')
# y_cols = ["target"]
# data = x_train
# data[y_cols] = y_train

In [ ]:
# num_cols = [
#        'sum_b_oper_3m', 'cnt_b_oper_3m', 'sum_c_oper_3m', 'cnt_c_oper_3m',
#        'sum_deb_d_oper_3m', 'cnt_deb_d_oper_3m', 'sum_cred_d_oper_3m',
#        'cnt_cred_d_oper_3m', 'sum_deb_e_oper_3m', 'cnt_deb_e_oper_3m',
#        'cnt_days_deb_e_oper_3m', 'sum_cred_e_oper_3m', 'cnt_cred_e_oper_3m',
#        'cnt_days_cred_e_oper_3m', 'sum_deb_f_oper_3m', 'cnt_deb_f_oper_3m',
#        'cnt_days_deb_f_oper_3m', 'sum_cred_f_oper_3m', 'cnt_cred_f_oper_3m',
#        'cnt_days_cred_f_oper_3m', 'sum_deb_g_oper_3m', 'cnt_deb_g_oper_3m',
#        'cnt_days_deb_g_oper_3m', 'sum_cred_g_oper_3m', 'cnt_cred_g_oper_3m',
#        'cnt_days_cred_g_oper_3m', 'sum_deb_h_oper_3m', 'cnt_deb_h_oper_3m',
#        'cnt_days_deb_h_oper_3m', 'sum_cred_h_oper_3m', 'cnt_cred_h_oper_3m',
#        'cnt_days_cred_h_oper_3m'
# ]

# cat_cols = [
#     "Sex",
#     "IsSeniorCitizen",
#     "HasPartner",
#     "HasChild",
#     "HasPhoneService",
#     "HasMultiplePhoneNumbers",
#     "HasInternetService",
#     "HasOnlineSecurityService",
#     "HasOnlineBackup",
#     "HasDeviceProtection",
#     "HasTechSupportAccess",
#     "HasOnlineTV",
#     "HasMovieSubscription",
#     "HasContractPhone",
#     "IsBillingPaperless",
#     "PaymentMethod",
# ]

# target = 'total_target'

In [ ]:
# train_df = pd.concat([train_df[num_cols], train_df[target]], axis = 1)
# train_df

In [ ]:
# train_df.dropna(inplace=True)

# Choose model

* Используем choos model из предыдущей работы - расширенный - https://www.kaggle.com/code/akscent/choice-of-boosting
* Отличный ноутбук с оценкой и ансамблированием моделей - https://www.kaggle.com/code/ldfreeman3/a-data-science-framework-to-achieve-99-accuracy
* Далее оптимизаторы гиперпараметр**ов - https://www.kaggle.com/code/akscent/ods-boosting
* Далее стекинг - https://www.kaggle.com/code/akscent/ods-boosting ; https://alexanderdyakonov.wordpress.com/2017/03/10/c%D1%82%D0%B5%D0%BA%D0%B8%D0%BD%D0%B3-stacking-%D0%B8-%D0%B1%D0%BB%D0%B5%D0%BD%D0%B4%D0%B8%D0%BD%D0%B3-blending/ ; https://github.com/a-milenkin/Competitive_Data_Science/blob/main/notebooks/6.3%20-%20Automatic%20Stacking.ipynb
* приемы в мл - https://github.com/Dyakonov/ml_hacks/tree/master

In [ ]:
# df_target_0 = train_df[train_df['total_target'] == 0].head(25000)
# df_target_1 = train_df[train_df['total_target'] == 1].head(25000)

# train_df = pd.concat([df_target_0, df_target_1], ignore_index=True)
# train_df = train_df.sample(frac=1, random_state=42).reset_index(drop=True)

# One-Hot-Encoder for city type

In [ ]:
test_df.isnull().sum().max()

In [ ]:
x_train

In [ ]:
object_columns = x_train.select_dtypes(include=['object']).columns
x_train[object_columns] = x_train[object_columns].astype('category')
test_df[object_columns] = test_df[object_columns].astype('category')

In [ ]:
# cat_cols = [
#     'channel_code',
#     'index_city_code', 'ogrn_month', 'ogrn_year',
#     'branch_code', 'okved', 'segment', 'city_type_1252', 'city_type_3597', 'city_type_other'
# ]
# type_city = ['city_type_1252', 'city_type_3597', 'city_type_other']

# cat_cols = list(x_train[object_columns].astype('category').columns)


In [ ]:
x_train.columns

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, classification_report

def evaluate_classification_metrics(y_true, y_pred, model_name):
    cm = confusion_matrix(y_true, y_pred)
    print(f"Classification Report for {model_name}:\n", classification_report(y_true, y_pred))
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=False)
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.title('Confusion Matrix')
    plt.show()


In [ ]:
def evaluate_classification_metrics(y_true, y_pred, model_name):
    """
    By class confusion matrix and report for other multi-class metrices
    """
    cm = confusion_matrix(y_true, y_pred)
    print(f"Classification Report for {model_name}:\n", classification_report(y_true, y_pred))
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=False)
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.title('Confusion Matrix')
    plt.show()

In [ ]:
#Common Model Algorithms
from sklearn import svm, tree, linear_model, neighbors, naive_bayes, ensemble, discriminant_analysis, gaussian_process
from catboost import CatBoostClassifier, Pool
from xgboost import XGBClassifier, DMatrix
from lightgbm import LGBMClassifier, train, Dataset

#Common Model Helpers
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn import feature_selection
from sklearn import model_selection
from sklearn import metrics

#Visualization
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.pylab as pylab
import seaborn as sns
from tqdm.notebook import tqdm

# Optimization Hyperparameters
import optuna
import torch
import xgboost as xgb
from optuna.samplers import TPESampler
from sklearn.metrics import f1_score, roc_auc_score
from sklearn.model_selection import StratifiedShuffleSplit

# Stacking 
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_predict

#%matplotlib inline = show plots in Jupyter Notebook browser
%matplotlib inline

In [ ]:
class EvalModels:
    """
    Class for evaluate binary classification models
    """
    
    def __init__(self, data, y_col, test_size=0.2, train_size=0.7, random_state=43):
        self.data = data
        self.y_col = y_col
        self.x_cols = list(data.drop(columns=y_col).columns)
        self.test_size = test_size
        self.train_size = train_size
        self.random_state = random_state
    
    def simple_eval(self, cv_splits=5):
        """
        data - pd.dataFrame
        y_col - list with target col name
        cv_split - split for cross validation
        test_size, train_size  - % of splitting
        random state - rand state for models
        """
        
        object_columns = self.data[self.x_cols].select_dtypes(include=['object']).columns
        cat_idx = [self.data.columns.get_loc(col) for col in object_columns.to_list()]
        if not object_columns.empty:
            self.data[object_columns] = self.data[object_columns].astype('category')

        # list with models. Стоит продолжить расширять.
        MLA = []

        if not object_columns.empty:
            MLA.append(XGBClassifier(
                verbose=-1, enable_categorical=True,
            ))
        else:
            MLA.append(XGBClassifier(
                verbose=-1,
                enable_categorical=False,
            ))

        if not object_columns.empty:
            MLA.append(LGBMClassifier(
                verbose=-1, cat_feature=cat_idx, 
                n_estimators=100,
                learning_rate=0.1,
                reg_alpha=0.5,
                reg_lambda=0.3,
                seed=42,
            ))
        else:
            MLA.append(LGBMClassifier(
                verbose=-1,
                n_estimators=100,
                learning_rate=0.1,
                reg_alpha=0.5,
                reg_lambda=0.3,
                seed=42,
            ))

        if not object_columns.empty:
            MLA.append(CatBoostClassifier(
                verbose=0, cat_features=object_columns,
                depth=4, iterations=3500, colsample_bylevel=0.098,
                subsample=0.95, l2_leaf_reg=9, min_data_in_leaf=243,
                max_bin=187,
            ))
        else:
            MLA.append(CatBoostClassifier(
                verbose=0, depth=4, iterations=3500,
                colsample_bylevel=0.098, subsample=0.95,
                l2_leaf_reg=9, min_data_in_leaf=243,
                max_bin=187,
            ))

        MLA_columns = ['MLA Name', 'MLA Parameters', 'MLA Train Accuracy Mean', 'MLA Test Accuracy Mean',
                       'MLA Test Accuracy 3*STD', 'MLA Time']
        MLA_compare = pd.DataFrame(columns=MLA_columns)

        cv_split = model_selection.StratifiedShuffleSplit(n_splits=cv_splits, test_size=test_size,
                                                          train_size=train_size, random_state=random_state)

        row_index = 0
        for alg in tqdm(MLA, total=len(MLA), desc="ML first estimate"):
            MLA_name = alg.__class__.__name__
            MLA_compare.loc[row_index, 'MLA Name'] = MLA_name
            MLA_compare.loc[row_index, 'MLA Parameters'] = str(alg.get_params())

            cv_results = model_selection.cross_validate(alg, self.data[self.x_cols], self.data[self.y_col], cv=cv_split,
                                                        return_train_score=True)
            print(f"{MLA_name} estimated: {cv_results['test_score'].mean()}")
            MLA_compare.loc[row_index, 'MLA Time'] = cv_results['fit_time'].mean()
            MLA_compare.loc[row_index, 'MLA Train Accuracy Mean'] = cv_results['train_score'].mean()
            MLA_compare.loc[row_index, 'MLA Test Accuracy Mean'] = cv_results['test_score'].mean()
            MLA_compare.loc[row_index, 'MLA Test Accuracy 3*STD'] = cv_results['test_score'].std() * 3

            row_index += 1

        MLA_compare.sort_values(by=['MLA Test Accuracy Mean'], ascending=False, inplace=True)
        list_top_models = MLA_compare.head(3)['MLA Name'].tolist()

        return MLA_compare, list_top_models


#     result, best_list = simple_eval(data, y_col)

    def fit_model(trial, X_train, y_train, X_valid, y_valid, model_name):

        if model_name == "CatBoostClassifier":
            params = {
                "iterations": trial.suggest_int("iterations", 500, 800),
                "learning_rate": trial.suggest_float("learning_rate", 0.005, 0.01),
                "auto_class_weights": trial.suggest_categorical("auto_class_weights", ["SqrtBalanced", "Balanced", "None"]),
                "depth": trial.suggest_int("depth", 3, 6),
                'loss_function':'Logloss',
                'use_best_model': True,
                'nan_mode': trial.suggest_categorical("nan_mode", ["Min", "Max"])}
            train_dataset = Pool(data=X_train, label=y_train,)
            eval_dataset = Pool(data=X_valid, label=y_valid, )
            clf = CatBoostClassifier(verbose = 0, random_seed = 41, **params)
            clf.fit(train_dataset, eval_set=eval_dataset, early_stopping_rounds=300)
        elif model_name == "LGBMClassifier":
            params = {'num_iterations': trial.suggest_int('num_iterations', 300, 500),
                'colsample_bytree': trial.suggest_float('colsample_bytree', 0.1, 0.5),
                'num_leaves': trial.suggest_int('num_leaves', 10, 20),
                'learning_rate': trial.suggest_float('learning_rate', 0.1, 0.5),
                'boosting_type': trial.suggest_categorical("boosting_type", ['gbdt', 'dart', 'goss']),
                'objective': 'binary',
                'metric': 'binary_logloss',
                'max_bin': trial.suggest_int('max_bin', 255, 4095),
                'force_col_wise': True,
                'is_unbalance': True}

            clf = LGBMClassifier(verbose = -1, random_seed = 42)
            clf = clf.set_params(**params)
            clf.fit(X_train, y_train)
        elif model_name == "XGBClassifier":
            params = {
                'subsample': trial.suggest_float('subsample', 0.6, 1.0),
                'n_estimators': trial.suggest_int('n_estimators', 300, 500),
                'eta': trial.suggest_float('eta', 0.01, 0.1),
                'min_child_weight': trial.suggest_float('min_child_weight', 1, 5),
                'colsample_bytree': trial.suggest_float('colsample_bytree', 0.1, 0.5),
                'objective': 'binary:logistic',
                'enable_categorical': True}
            clf = XGBClassifier(tree_method = 'hist', device= 'cuda') if gpu_available else XGBClassifier(verbose = -1, random_seed = 43)
            clf = clf.set_params(**params)
            clf.fit(X_train, y_train)
        else:
            raise ValueError("Invalid model_name")
        y_pred = clf.predict_proba(X_valid)[:, 1]
        return clf, y_pred


    def objective(self, trial, model_name, return_models=False):
        sss = StratifiedShuffleSplit(n_splits=self.n_split, test_size=self.test_size, random_state=self.random_state)
        scores, models = [], []

        for train_idx, valid_idx in sss.split(self.data[self.x_cols], self.data[self.y_col]):
            X_train, X_valid = self.data[self.x_cols].iloc[train_idx, :], self.data[self.x_cols].iloc[valid_idx, :]
            y_train, y_valid = self.data[self.y_col].iloc[train_idx], self.data[self.y_col].iloc[valid_idx]

            X_train.columns = X_train.columns.str.replace(' ', '_')
            X_valid.columns = X_valid.columns.str.replace(' ', '_')

            clf, y_pred = self.fit_model(trial, X_train, y_train, X_valid, y_valid, model_name)
            roc_auc = roc_auc_score(y_valid, y_pred)
            scores.append(roc_auc)
            models.append(clf)
            break

        if return_models:
            return np.mean(scores), models
        else:
            return np.mean(scores)

    def optimize_models(self):
        _, list_best_models = self.simple_eval()

        optimization_results, best_score = [], []
        for model_name in tqdm(list_best_models, total=len(list_best_models), desc='Optimizing models'):
            if model_name in list_best_models:
                study = optuna.create_study(direction='maximize', sampler=TPESampler(seed=10))
                study.optimize(lambda trial: self.objective(trial, model_name), n_trials=50, n_jobs=-1, show_progress_bar=True)
                best_model_params = study.best_params
                valid_scores, model = self.objective(optuna.trial.FixedTrial(study.best_params), model_name, return_models=True)

                result = {
                    'model_name': model_name,
                    'num_trials': len(study.trials),
                    'best_trial_value': valid_scores,
                    'best_trial_params': best_model_params,
                }   
                optimization_results.append(result)
                best_score.append(valid_scores)
                print(f'Result for {model_name}: {valid_scores}')
            else:
                continue


        print("Optimization results:")
        for result in optimization_results:
            print(f"Model: {result['model_name']}, Num Trials: {result['num_trials']}, Best Trial Value: {result['best_trial_value']}")
            print(f"Best Trial Params: {result['best_trial_params']}")
            print("\n")
                
        return optimization_results, best_score


In [ ]:
model_ev = EvalModels(data, y_col)

In [ ]:
optimization_results, best_score = model_ev.optimize_models()

In [ ]:
result.head(3)['MLA Name'].tolist()

In [ ]:
result['MLA Parameters']

In [ ]:
# result["MLA Test Accuracy Mean"] = result["MLA Test Accuracy Mean"].astype(float)
# list_best_models = list(result.nlargest(3, 'MLA Test Accuracy Mean')["MLA Name"])

In [ ]:
list_best_models = ['XGBClassifier', 'CatBoostClassifier', 'LGBMClassifier']

# Search hyperparams

In [ ]:
for result in optimization_results:
    model_name = result['model_name']
    best_params = result['best_trial_params']

    if model_name == 'LGBMClassifier':
        model = LGBMClassifier(**best_params)
    elif model_name == 'XGBClassifier':
        model = XGBClassifier(**best_params)
    elif model_name == 'CatBoostClassifier':
        model = CatBoostClassifier(**best_params)
    else:
        print(f"Unsupported model: {model_name}")
        continue

    # Assuming you have a function train_model() for training the model
    trained_model = train_model(model, self.data[self.x_cols], self.data[self.y_col])

    # Assuming you have a function evaluate_model() for evaluating the model
    evaluation_result = evaluate_model(trained_model, self.data[self.x_cols], self.data[self.y_col])

    print(f"Model: {model_name}, Evaluation Result: {evaluation_result}")


In [ ]:
x_train.select_dtypes(include=['category']).columns

In [ ]:
x_train[x_cols]

In [ ]:
#     # Список гиперпараметров для каждой модели
#     MLA_params = [
#         {'name': 'AdaBoost', 'params': {'n_estimators': trial.suggest_int('n_estimators', 2000, 5000), 'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.2),}},
#     #     {'name': 'BaggingClassifier', 'params': {'n_estimators': [10, 50, 100], 'max_samples': [0.5, 0.7, 1.0]}},
#     #     {'name': 'ExtraTrees', 'params': {'n_estimators': [50, 100, 200], 'max_features': ['auto', 'sqrt', 'log2']}},
#     #     {'name': 'GradientBoosting', 'params': {'n_estimators': [50, 100, 200], 'learning_rate': [0.01, 0.1, 0.2], 'max_depth': [3, 5, 7]}},
#     #     {'name': 'RandomForest', 'params': {'n_estimators': [50, 100, 200], 'max_features': ['auto', 'sqrt', 'log2']}},
#     #     {'name': 'LogisticRegressionCV', 'params': {'Cs': [1, 10, 100], 'penalty': ['l1', 'l2']}},
#     #     {'name': 'PassiveAggressiveClassifier', 'params': {'C': [0.1, 1, 10], 'max_iter': [100, 200, 500]}},
#     #     {'name': 'RidgeClassifierCV', 'params': {'alphas': [(1e-3, 1e-2, 1e-1), (1, 10, 100)]}},
#     #     {'name': 'SGDClassifier', 'params': {'alpha': [1e-3, 1e-2, 1e-1], 'max_iter': [100, 200, 500]}},
#     #     {'name': 'Perceptron', 'params': {'alpha': [1e-3, 1e-2, 1e-1], 'max_iter': [100, 200, 500]}},
#     #     {'name': 'BernoulliNB', 'params': {'alpha': [0.1, 1, 2], 'binarize': [0.0, 0.5, 1.0]}},
#     #     {'name': 'GaussianNB', 'params': {}},
#         {'name': 'KNeighborsClassifier', 'params': {'n_neighbors': trial.suggest_int('n_neighbors',3, 7), 
#                                                     'weights': trial.suggest_categorical("weights", ['uniform', 'distance'])}},
#     #     {'name': 'SVC', 'params': {'C': [0.1, 1, 10], 'kernel': ['linear', 'rbf']}},
#     #     {'name': 'NuSVC', 'params': {'nu': [0.1, 0.3, 0.5], 'kernel': ['linear', 'rbf']}},
#     #     {'name': 'LinearSVC', 'params': {'C': [0.1, 1, 10], 'penalty': ['l1', 'l2']}},
#     #     {'name': 'DecisionTreeClassifier', 'params': {'criterion': ['gini', 'entropy'], 'max_depth': [3, 5, 7]}},
#         {'name': 'ExtraTreesClassifier', 'params': {'criterion': trial.suggest_categorical("criterion", ['gini', 'entropy']), 
#                                                    'max_depth': trial.suggest_categorical('max_depth', [3, 7])}},

#     #     {'name': 'LinearDiscriminantAnalysis', 'params': {}},
#     #     {'name': 'QuadraticDiscriminantAnalysis', 'params': {}},
#         {'name': 'XGBClassifier', 'params': {
#             'subsample': trial.suggest_float('subsample', 0.6, 1.0),
#             'n_estimators': trial.suggest_int('n_estimators', 300, 500),
#             'eta': trial.suggest_float('eta', 0.01, 0.1),
#             'booster': trial.suggest_categorical("booster", ['gbdt', 'dart', 'goss']),
#             'min_child_weight': trial.suggest_int('min_child_weight', 1, 5),
#             'colsample_bytree': trial.suggest_float('colsample_bytree', 0.1, 0.5),
#             'objective': 'binary:logistic',}},
#         {'name': 'CatBoostClassifier', 'params': {
#             "iterations": trial.suggest_int("iterations", 500, 800),
#             "learning_rate": trial.suggest_float("learning_rate", 0.005, 0.01),
#             "auto_class_weights": trial.suggest_categorical("auto_class_weights", ["SqrtBalanced", "Balanced", "None"]),
#             "depth": trial.suggest_int("depth", 3, 6),
#             'loss_function':'Logloss',
#             'use_best_model': True,
#             'nan_mode': trial.suggest_categorical("nan_mode", ["Min", "Max"]),
#         }},
#         {'name': 'LGBMClassifier', 'params': {'num_iterations': trial.suggest_int('num_iterations', 300, 500),
#             'colsample_bytree': trial.suggest_float('colsample_bytree', 0.1, 0.5),
#             'num_leaves': trial.suggest_int('num_leaves', 10, 20),
#             'learning_rate': trial.suggest_float('learning_rate', 0.1, 0.5),
#             'boosting_type': trial.suggest_categorical("boosting_type", ['gbdt', 'dart', 'goss']),
#             'objective': 'binary',
#             'metric': 'binary_logloss',
#             'force_col_wise': True,
#             'is_unbalance': True}}
#     ]
    
#     available_model_names = [model['name'] for model in MLA_params]
#     if model_name not in available_model_names:
#         print(f"Error: Model name '{model_name}' not found in available models. Please check the model name.")
#     else:
#         filtered_MLA_params = None
#         for model in MLA_params:
#             if model['name'] == model_name:
#                 filtered_MLA_params = model['params']
#                 break
#             else:
#                 continue

#         if not filtered_MLA_params:
#             print(f"Warning: No parameters found for the specified model '{model_name}'.")
#         else:
#             if not filtered_MLA_params:
#                 print(f"Model {model_name} not found in MLA_params. Skipping optimization.")
#                 return 0

In [ ]:
import optuna
import torch
import xgboost as xgb
from optuna.samplers import TPESampler
from sklearn.metrics import f1_score, roc_auc_score
from sklearn.model_selection import StratifiedShuffleSplit
from tqdm import tqdm


# column_numbers = [5, 6, 9, 10, 11, 16, 17, 88, 89, 90]
# x_train[type_city] = x_train[type_city].astype('str')
# test_df[type_city] = test_df[type_city].astype('str')
# for col in cat_cols:
#     x_train[col] = pd.Categorical(x_train[col])
#     test_df[col] = pd.Categorical(test_df[col])
            
def fit_MLA(trial, X_train, y_train, X_valid, y_valid, MLA, model_name):
            
    if model_name == "CatBoostClassifier":
        params = {
            "iterations": trial.suggest_int("iterations", 500, 800),
            "learning_rate": trial.suggest_float("learning_rate", 0.005, 0.01),
            "auto_class_weights": trial.suggest_categorical("auto_class_weights", ["SqrtBalanced", "Balanced", "None"]),
            "depth": trial.suggest_int("depth", 3, 6),
            'loss_function':'Logloss',
            'use_best_model': True,
            'nan_mode': trial.suggest_categorical("nan_mode", ["Min", "Max"])}
        train_dataset = Pool(data=X_train, label=y_train,)
        eval_dataset = Pool(data=X_valid, label=y_valid, )
        clf = CatBoostClassifier(verbose = 0, random_seed = 41, **params)
        clf.fit(train_dataset, eval_set=eval_dataset, early_stopping_rounds=300)
    elif model_name == "LGBMClassifier":
        params = {'num_iterations': trial.suggest_int('num_iterations', 300, 500),
            'colsample_bytree': trial.suggest_float('colsample_bytree', 0.1, 0.5),
            'num_leaves': trial.suggest_int('num_leaves', 10, 20),
            'learning_rate': trial.suggest_float('learning_rate', 0.1, 0.5),
            'boosting_type': trial.suggest_categorical("boosting_type", ['gbdt', 'dart', 'goss']),
            'objective': 'binary',
            'metric': 'binary_logloss',
            'max_bin': trial.suggest_int('max_bin', 255, 4095),
            'force_col_wise': True,
            'is_unbalance': True}
        
        clf = LGBMClassifier(verbose = -1, random_seed = 42)
        clf = clf.set_params(**params)
        clf.fit(X_train, y_train)
    elif model_name == "XGBClassifier":
        params = {
            'subsample': trial.suggest_float('subsample', 0.6, 1.0),
            'n_estimators': trial.suggest_int('n_estimators', 300, 500),
            'eta': trial.suggest_float('eta', 0.01, 0.1),
            'min_child_weight': trial.suggest_float('min_child_weight', 1, 5),
            'colsample_bytree': trial.suggest_float('colsample_bytree', 0.1, 0.5),
            'objective': 'binary:logistic',
            'enable_categorical': True}
        clf = XGBClassifier(tree_method = 'hist', device= 'cuda') if gpu_available else XGBClassifier(verbose = -1, random_seed = 43)
        clf = clf.set_params(**params)
        clf.fit(X_train, y_train)
    else:
        params = {
            'n_estimators': trial.suggest_int('n_estimators', 500, 2000),
            'max_depth': trial.suggest_int('max_depth', 3, 20),
            'min_samples_split': trial.suggest_float('min_samples_split', 0.1, 1.0),
            'min_samples_leaf': trial.suggest_float('min_samples_leaf', 0.1, 0.5),
            'class_weight': trial.suggest_categorical('class_weight', ['balanced', 'balanced_subsample']),}
        clf = MLA
        clf = clf.set_params( **params)
        clf.fit(X_train, y_train)
    y_pred = clf.predict_proba(X_valid)[:, 1]
    return clf, y_pred

# Определение функции objective
def objective(trial, return_models=False):
    
    n_splits = 5
    sss = StratifiedShuffleSplit(n_splits=n_splits, test_size=0.2, random_state=42)
    scores, models = [], []

    for train_idx, valid_idx in sss.split(data[x_cols], data[y_col]):
        X_train, X_valid = data[x_cols].iloc[train_idx, :], data[x_cols].iloc[valid_idx, :]
        y_train, y_valid = data[y_col].iloc[train_idx], data[y_col].iloc[valid_idx]

        X_train.columns = X_train.columns.str.replace(' ', '_')
        X_valid.columns = X_valid.columns.str.replace(' ', '_')

        clf, y_pred = fit_MLA(trial, X_train, y_train, X_valid, y_valid, model, model_name)
        roc_auc = roc_auc_score(y_valid, y_pred)
        scores.append(roc_auc)
        models.append(clf)
        break
        
    if return_models:
        return np.mean(scores), models
    else:
        return np.mean(scores)


def optim(objective, list_best_models):
    # Список для сохранения результатов оптимизации по моделям
    optimization_results, optimization_models = [], []
    # Оптимизация гиперпараметров для каждой модели
    for model in tqdm(list_best_models, total=len(list_best_models), desc='Optimizing models'):
        model_name = model
        if model_name in list_best_models:
        #     result, clm = optimize_model(x_train[x_cols], x_train[y_cols], model_name, model)
            study = optuna.create_study(direction='maximize', sampler=TPESampler())
        #     study.optimize(lambda trial: objective(trial, X, y, MLA, model_name), n_trials=5)
            study.optimize(objective, n_trials=50, n_jobs = -1, show_progress_bar=True,)
            best_model_params = study.best_params
            model, valid_scores = objective(optuna.trial.FixedTrial(study.best_params), return_models=True)

            result = {
                'model_name': model_name,
                'num_trials': len(study.trials),
                'best_trial_value': valid_scores,
                'best_trial_params': best_model_params,
            }   
            optimization_results.append(result)
            optimization_models.append(model)
            print(f'Result for {model_name}: {valid_scores}')
        else:
            continue
            
    return optimization_results, optimization_models

optimization_results, optimization_models = optim(objective, list_best_models)
print("Optimization results:")
for result in optimization_results:
    print(f"Model: {result['model_name']}, Num Trials: {result['num_trials']}, Best Trial Value: {result['best_trial_value']}")
    print(f"Best Trial Params: {result['best_trial_params']}")
    print("\n")

In [ ]:
optimization_results, optimization_models

In [ ]:
trained_models = []  # Список для хранения обученных моделей

for result in optimization_results:
    for model in MLA:
        if model.__class__.__name__ == result['model_name']:
            valid_params = model.get_params()
            filtered_params = {k: v for k, v in result['best_trial_params'].items() if k in valid_params}
            
            # Создаем новый экземпляр модели с лучшими гиперпараметрами
            if result['model_name'] == "CatBoostClassifier":
                model_instance = CatBoostClassifier(**filtered_params)
            else:
                model_instance = model.set_params(**filtered_params)
            
            trained_models.append(model_instance)
            

In [ ]:
trained_models

In [ ]:
from sklearn.metrics import roc_auc_score

def print_roca_auc(clf, X, y, label):
    a = clf.predict(X)
    
    print(label + ' AUC-ROC  = ' + str(roc_auc_score(y, a)))

In [ ]:
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_predict

class DjStacking(BaseEstimator, ClassifierMixin):  
    """Стэкинг моделей scikit-learn"""

    def __init__(self, models, ens_model):
        """
        Инициализация
        models - базовые модели для стекинга
        ens_model - мета-модель
        """
        self.models = models
        self.ens_model = ens_model
        self.n = len(models)
        self.valid = None
        
    def fit(self, X, y=None, p=0.25, cv=5, err=0.001, random_state=None):
        """
        Обучение стекинга
        p - в каком отношении делить на обучение / тест
            если p = 0 - используем всё обучение!
        cv  (при p=0) - сколько фолдов использовать
        err (при p=0) - величина случайной добавки к метапризнакам
        random_state - инициализация генератора
            
        """
        if (p > 0): # делим на обучение и тест
            # разбиение на обучение моделей и метамодели
            train, valid, y_train, y_valid = train_test_split(X, y, test_size=p, random_state=random_state)
            
            # заполнение матрицы для обучения метамодели
            self.valid = np.zeros((valid.shape[0], self.n))
            for t, clf in enumerate(self.models):
                clf.fit(train, y_train)
                self.valid[:, t] = clf.predict_proba(valid)[:,1]
                
            # обучение метамодели
            self.ens_model.fit(self.valid, y_valid)
            
        else: # используем всё обучение
            
            # для регуляризации - берём случайные добавки
            self.valid = err*np.random.randn(X.shape[0], self.n)
            
            for t, clf in enumerate(self.models):
                # это oob-ответы алгоритмов
                self.valid[:, t] += cross_val_predict(clf, X, y, cv=cv, n_jobs=-1, method='predict')
                # но сам алгоритм надо настроить
                clf.fit(X, y)
            
            # обучение метамодели
            self.ens_model.fit(self.valid, y)  
            

        return self
    


    def predict(self, X, y=None):
        """
        Работа стэкинга
        """
        # заполение матрицы для мета-классификатора
        X_meta = np.zeros((X.shape[0], self.n))
        
        for t, clf in enumerate(self.models):
            X_meta[:, t] = clf.predict_proba(X)[:, 1]
        
        a = self.ens_model.predict_proba(X_meta)[:, 1]
        
        return (a)

In [ ]:
from sklearn.linear_model import Ridge

train_X, test_X, train_y, test_y = train_test_split(x_train, y, train_size=0.8, 
                                                    random_state=1999, stratify=x_train[y_cols])
ens_model = Ridge()
s1 = DjStacking(trained_models, ens_model)
s1.fit(train_X, train_y)

# model_name = f'{run.name}_model.mod'
# bstr = s1.booster_
# bstr.save_model(model_name)
# config = s1.get_params()
# model_art = wandb.Artifact(name=model_name, type='model', metadata=config)
# model_art.add_file(model_name)
# run.log_artifact(model_art)

# run.summary["best_score"] = bstr.best_score
# run.summary["best_iteration"] = bstr.best_iteration
# preds = model.predict(x_val)
# run.summary["f1_score"] = f1_score(y_val, preds, average='macro')
# run.summary["accuracy"] = accuracy_score(y_val, preds)

# model_name = f'{run.name}_model.mod'
# bstr = s2.booster_
# bstr.save_model(model_name)
# config = s2.get_params()
# model_art = wandb.Artifact(name=model_name, type='model', metadata=config)
# model_art.add_file(model_name)
# run.log_artifact(model_art)

# run.summary["best_score"] = bstr.best_score
# run.summary["best_iteration"] = bstr.best_iteration
# preds = model.predict(x_val)
# run.summary["f1_score"] = f1_score(y_val, preds, average='macro')
# run.summary["accuracy"] = accuracy_score(y_val, preds)

# run.finish()

In [ ]:
print_roca_auc(s1, test_X, test_y, '1-stacking')

In [ ]:
s1.predict(test_df)

In [ ]:
# test_df = test_df.astype({'channel_code': 'category', 'branch_code': 'category'})

In [ ]:
# s1 = DjStacking(trained_models, ens_model)
# a = 0
# e = []
# for t in range(2, 11):
#     s1.fit(train_X, train_y, p=-1, cv=t, err=0.00)
#     a = s1.predict(test_X, train_y)
#     auc = roc_auc_score(test_y, a)
#     print (auc)
#     e.append(auc)

In [ ]:
sample_submission_df = pd.read_csv("/kaggle/input/sibalfahack/Siberian Alfa Hack Materials/Siberian Alfa Hack Materials/sample_submission.csv")

for i in trained_models:
    if i.__class__.__name__ == CatBoostClassifier:
        
        score = i.predict_proba(test_df)[:, 1]
        sample_submission_df["score"] = score
        sample_submission_df.to_csv(f"my_submission_{i}.csv", index=False)
    else:
        clf = i
        clf.fit(x_train[x_cols], x_train[y_cols])
        score = i.predict_proba(test_df)[:, 1]
        sample_submission_df["score"] = score
        sample_submission_df.to_csv(f"my_submission_{i}.csv", index=False)
    
# score = s1.predict(test_df)
# sample_submission_df["score"] = score
# sample_submission_df.to_csv("my_submission_essemble.csv", index=False)
    

# Pipeline for ensemble